In [1]:
import pandas as pd
import matplotlib.pyplot as plt

match = "/content/drive/MyDrive/LATIHAN PYHTON/output.csv"
match_new = "/content/drive/MyDrive/LATIHAN PYHTON/output_new.csv"
df1 = pd.read_csv(match, index_col=False)
df2 = pd.read_csv(match_new, index_col=False)
df2 = df2.replace(2022,2023)
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
df = df1.append(df2, ignore_index=True)

In [2]:
matches = df
matches["date"] = pd.to_datetime(matches["date"])
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+","", regex=True).astype("int")
matches["referee"] = matches["referee"].astype("category").cat.codes
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["result"] == "W").astype("int")
matches['xga'] = matches['xga'].apply(lambda x: x*-1 if x > 0 else x)
matches['ga'] = matches['ga'].apply(lambda x: x*-1 if x > 0 else x)
matches["g-xg"] = matches['gf'] - matches['xg']

#rolling averages untuk mengetahui performa tim di 3 laga terakhir
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3, closed='left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group

cols = ["xg","xga","gf","ga","sh","sot","dist","fk","g-xg"]
new_cols = [f'{c}_rolling' for c in cols]

matches_rolling=matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel("team")
matches_rolling.index = range(matches_rolling.shape[0])


In [3]:
predictors = ["venue_code","opp_code","day_code"]
predictors = predictors+new_cols

#train & test dataset
train = matches_rolling[matches_rolling["date"] <= '2022-05-23']
test = matches_rolling[matches_rolling["date"] > '2022-05-23']

In [4]:
matches_rolling.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,g-xg,xg_rolling,xga_rolling,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,g-xg_rolling
0,2019-09-01,16:30,Premier League,Matchweek 4,Sun,Home,D,2.0,-2.0,Tottenham,...,-0.4,0.966667,-1.466667,1.333333,-1.333333,10.666667,4.666667,17.2,0.333333,0.366667
1,2019-09-15,16:30,Premier League,Matchweek 5,Sun,Away,D,2.0,-2.0,Watford,...,1.2,1.400000,-2.000000,1.666667,-2.000000,16.666667,6.666667,18.6,1.000000,0.266667
2,2019-09-22,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,-2.0,Aston Villa,...,0.6,1.400000,-2.400000,1.666667,-2.333333,14.000000,5.000000,19.3,1.000000,0.266667
3,2019-09-30,20:00,Premier League,Matchweek 7,Mon,Away,D,1.0,-1.0,Manchester Utd,...,-0.9,1.866667,-2.066667,2.333333,-2.000000,17.666667,5.666667,18.6,1.333333,0.466667
4,2019-10-06,14:00,Premier League,Matchweek 8,Sun,Home,W,1.0,0.0,Bournemouth,...,-0.2,1.700000,-1.700000,2.000000,-1.666667,12.333333,4.666667,18.1,1.000000,0.300000


In [5]:
import tensorflow as tf
import numpy as np

# Define the data set
# Inputs
# predictors
# Outputs
# match_rolling["target"]

# Define the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy',tf.keras.metrics.Precision()])

# Train the model
# inputs = np.concatenate((home_team, away_team), axis=1)
model.fit(train[predictors], train["target"], epochs=100, batch_size=32)

# Make predictions
# test_input = np.array([[1, 0], [0, 1]])
# test_input = np.concatenate((test_input, test_input), axis=1)
predictions = model.predict(test[predictors])

Epoch 1/100
69/69 [==============================] - 3s 7ms/step - loss: 0.7429 - accuracy: 0.5705 - precision: 0.3797
Epoch 2/100
69/69 [==============================] - 0s 7ms/step - loss: 0.6591 - accuracy: 0.6195 - precision: 0.5157
Epoch 3/100
69/69 [==============================] - 0s 6ms/step - loss: 0.6522 - accuracy: 0.6363 - precision: 0.5736
Epoch 4/100
69/69 [==============================] - 1s 8ms/step - loss: 0.6529 - accuracy: 0.6327 - precision: 0.5571
Epoch 5/100
69/69 [==============================] - 1s 7ms/step - loss: 0.6523 - accuracy: 0.6372 - precision: 0.5590
Epoch 6/100
69/69 [==============================] - 0s 5ms/step - loss: 0.6452 - accuracy: 0.6358 - precision: 0.5723
Epoch 7/100
69/69 [==============================] - 0s 7ms/step - loss: 0.6526 - accuracy: 0.6222 - precision: 0.5190
Epoch 8/100
69/69 [==============================] - 0s 7ms/step - loss: 0.6460 - accuracy: 0.6395 - precision: 0.5741
Epoch 9/100
69/69 [=============================

In [6]:
predictions = predictions.flatten()

In [7]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=predictions), index=test.index)

In [8]:
combined = combined.merge(matches_rolling[["date","team","opponent","result"]], left_index=True, right_index=True)

In [9]:
import numpy as np

combined["predicted"] = np.where(combined["predicted"]>0.5, 1, 0)

In [10]:
combined

,actual,predicted,date,team,opponent,result
111,1,0,2022-08-05,Arsenal,Crystal Palace,W
112,1,0,2022-08-13,Arsenal,Leicester City,W
113,1,1,2022-08-20,Arsenal,Bournemouth,W
114,1,1,2022-08-27,Arsenal,Fulham,W
115,1,1,2022-08-31,Arsenal,Aston Villa,W
...,...,...,...,...,...,...
2575,0,1,2022-11-12,Wolverhampton Wanderers,Arsenal,L
2576,1,0,2022-12-26,Wolverhampton Wanderers,Everton,W
2577,0,0,2022-12-31,Wolverhampton Wanderers,Manchester Utd,L
2578,0,0,2023-01-04,Wolverhampton Wanderers,Aston Villa,D


In [11]:
from sklearn.metrics import classification_report
print(classification_report(combined["actual"],combined["predicted"]))

              precision    recall  f1-score   support

           0       0.65      0.77      0.71       229
           1       0.50      0.36      0.41       146

    accuracy                           0.61       375
   macro avg       0.57      0.56      0.56       375
weighted avg       0.59      0.61      0.59       375

